# take2

**GOAL**: Keep It Stupid Simple = KISS.

**References**:
 1. [ilqgames/python](https://github.com/HJReachability/ilqgames/blob/master/python)

In [5]:
%load_ext autoreload
%autoreload 1
%aimport decentralized.control2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import functools

import numpy as np
import matplotlib.pyplot as plt
import torch

from decentralized import control2
import pocketknives

## dynamics

In [7]:
def unicycle_continuous(x, u):
    """
    Compute the time derivative of state for a particular state/control.
    NOTE: `x` and `u` should be 2D (i.e. column vectors).
    REF: [1]
    """
    assert isinstance(x, torch.Tensor) and isinstance(u, torch.Tensor)

    x_dot = torch.zeros((x.numel(), 1))
    x_dot[0, 0] = x[3, 0] * torch.cos(x[2, 0])
    x_dot[1, 0] = x[3, 0] * torch.sin(x[2, 0])
    x_dot[2, 0] = u[0, 0]
    x_dot[3, 0] = u[1, 0]
    return x_dot

## costs

In [8]:
def QR_goal_cost(x, u, _x_goal, _Q=None, _R=None, terminal=False):
    """Cost of reaching the goal"""
    
    assert isinstance(x, torch.Tensor) and isinstance(u, torch.Tensor)
    
    TERMINAL_WEIGHT = 1000.0    
    if _Q is None:
        _Q = torch.eye(x.numel())
    if _R is None:
        _R = torch.eye(u.numel())
    
    cost = (x - _x_goal).T @ _Q @ (x - _x_goal) + u.T @ _R @ u
    
    if terminal:
        return TERMINAL_WEIGHT * cost
    return cost

In [19]:
class network(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = torch.nn.Linear(4, 2)
        self.fc2 = torch.nn.Linear(2, 1)

In [20]:
network()

network(
  (fc1): Linear(in_features=4, out_features=2, bias=True)
  (fc2): Linear(in_features=2, out_features=1, bias=True)
)

In [13]:
f = torch.nn.Module()

In [9]:
dt = 0.1
N = 10
x = torch.tensor([[0, 0, 0, 1]], dtype=torch.float, requires_grad=True).T
u = torch.tensor([[0, 0]], dtype=torch.float, requires_grad=True).T
x_goal = torch.zeros((4, 1), dtype=torch.float)

goal_cost = functools.partial(QR_goal_cost, _x_goal=x_goal)

In [23]:
iLQR = control2.iLQR(unicycle_continuous, goal_cost, x.numel(), u.numel(), dt, N)

In [25]:
iLQR.solve(x)

RuntimeError: expand(torch.FloatTensor{[4, 1]}, size=[4]): the number of sizes provided (1) must be greater or equal to the number of dimensions in the tensor (2)

In [26]:
%debug

> /home/zjw/documents/grad/Research/decentralized/decentralized/control2.py(134)_rollout()
    132         N = U.shape[0]
    133         X = torch.zeros((N+1, self.n_x))
--> 134         X[0] = x0
    135         J = 0.0
    136 



ipdb>  q
